In [ ]:
!pip install stanza
import stanza
stanza.download('en')
stanza_nlp = stanza.Pipeline(lang = 'en', processors = 'tokenize,pos,constituency')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 32.0 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement graphiz (from versions: none)
ERROR: No matching distribution found for graphiz


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Downloaded file to /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor    | Package             |
--------------------------------------
| tokenize     | combined            |
| mwt          | combined            |
| pos          | combined_charlm     |
| constituency | ptb3-revised_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
/usr/local/lib/python3.10/dist-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpick

In [3]:
import pandas as pd
import os
import re
from tqdm import tqdm
import spacy
nlp = spacy.load('en_core_web_sm')
tqdm.pandas()

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

indexes_train = list(train.index.values)
indexes_test = list(test.index.values)
train['id'] = indexes_train
test['id'] = indexes_test

frames = [train, test]
merged = pd.concat(frames)
sarcasm_rows = merged.loc[merged['class'] == 'sarcasm']
regular_rows = merged.loc[merged['class'] == 'regular']
all_rows = pd.concat([sarcasm_rows, regular_rows])
all_rows["sarcastic"] = all_rows.apply( lambda x : int(x["class"] == 'sarcasm'),  axis=1 )
all_rows = all_rows.drop( ["class"], axis=1 )
all_rows = all_rows.rename(columns={"tweets" : "original_tweet"})

In [5]:
hate = re.compile(r'([\b]+[Hh]+[Aa]+[Tt]+)')
love = re.compile(r'([Ll]+[OoUu]+[Vv]+)')

all_rows["love/hate"] = all_rows.apply( lambda x : int( re.search(love, x["original_tweet"]) is not None or re.search(hate, x["original_tweet"] ) is not None ), axis=1 )

In [6]:
only_love_hate = all_rows[ all_rows["love/hate"] == 1]

In [ ]:
sarc = re.compile(r'([#]+[Ss]+[Aa]+[Rr]+[Cc]+[^\s]*)')
attr = re.compile(r'([@]+[^\s]*)')

# applying to the original tweet the following substitutions : replaced the @... by @user, removed the #sarcasm, limited to the maximum number of adjacent blankspaces to 1
only_love_hate["tweet"] = only_love_hate.apply( lambda x : re.sub( ' +', ' ', re.sub( sarc, '', re.sub( attr, "@user", x["original_tweet"] ) ) ), axis = 1 )

<ipython-input-7-3eef9d446850>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_love_hate["tweet"] = only_love_hate.apply( lambda x : re.sub( ' +', ' ', re.sub( sarc, '', re.sub( attr, "@user", x["original_tweet"] ) ) ), axis = 1 )


## Filtering

In [ ]:
doc = stanza_nlp("i love work today :) .")

# write a function that looks at each layer, and checks, if there is an SBAR, if it is a correct SBAR or not
def parse_vp(tree, left_sister_is_verb=False, is_embedded=False, first_VP_passed=False ):

    if 'SBAR' == tree.label:
      if 'S' == tree.children[0].label and not left_sister_is_verb :
        return 'bad'
      else:
        return parse_vp(tree.children[-1], left_sister_is_verb= len(tree.children) > 1 and tree.children[-2].label.startswith("V"), is_embedded = True, first_VP_passed= first_VP_passed)

    if tree.is_preterminal():
      return 'good'

    if tree.label == "VP" and not first_VP_passed :
      first_VP_passed = True
      if len( [x for x in tree.children if x.label == "VP"] ) >= 2 :
        return 'bad'

    if tree.label == "UH" and is_embedded :
      return 'bad'

    if tree.label == 'NP' and len(tree.children) >= 2 and tree.children[0].label == 'NP' and tree.children[1].label == 'SBAR'  :
      x = parse_vp(tree.children[1].children[0], is_embedded=True, first_VP_passed = first_VP_passed)
      if x == 'bad' :
        return x
      for i in range(len(tree.children) - 2) :
        idx = len(tree.children) - 1 - i
        child = tree.children[idx]
        x = parse_vp(child, left_sister_is_verb = idx > 0 and tree.children[idx - 1].label.startswith("V"), is_embedded = is_embedded, first_VP_passed = first_VP_passed)
        if x == 'bad':
          return x
      return 'good'

    ignore_S = False
    if 'CC' in [ch.label for ch in tree.children[1:]] :
      if tree.label == 'S' and not is_embedded :
        return 'bad'
      else :
        ignore_S = True

    for i in range(len(tree.children)):
      idx = len(tree.children) - i - 1
      child = tree.children[idx]

      if 'S' == child.label and (not ignore_S) and idx > 0 and not tree.children[0].label.startswith('V') and not tree.children[idx - 1].label.startswith('W') and not tree.children[idx-1].label == "IN" :
        return 'bad'

      x = parse_vp(child, left_sister_is_verb = idx > 0 and tree.children[idx - 1].label.startswith("V"), is_embedded = is_embedded and (tree.label == "S"), first_VP_passed=first_VP_passed)
      if x == 'bad':
        return x
    return 'good'

(ROOT (S (NP (PRP i)) (VP (VBP love) (NP (NN work)) (NP (NN today)) (NFP :-RRB-)) (. .)))


In [ ]:
def filtering(s):
  doc = stanza_nlp(s)
  if len(doc.sentences) == 1:
    return parse_vp(doc.sentences[0].constituency)
  else:
    return 'bad'

'bad'

In [ ]:
def capitalize_I(text) :
  lower_case_I = re.compile(r'\b[i]\b')
  return re.sub(lower_case_I, 'I', text)

users = re.compile(r'([@]+\w*\s)')
def preprocess_tweet(text) :
  return str(re.sub(users, '', capitalize_I(text)))

In [11]:
def exp_1_filtered(s):
  s = preprocess_tweet(s)
  if (filtering(s) == 'good'):
    doc = nlp(s)
    for i in range(len(doc)):
      if doc[i].pos_ == 'VERB':
        if (i != 0) and (doc[i - 1].pos_ != 'ADV') and (doc[i].lemma_ == 'love' or doc[i].lemma_ == 'hate') and [x for x in doc[i].children if x.text.lower() == 'i']:
          if doc[i].lemma_ == 'love':
            return 'love'
          else:
            return 'hate'
        else:
          return None

In [12]:
def exp_2_filtered(s):
  s = preprocess_tweet(s)
  if (filtering(s) == 'good'):
    doc = nlp(s)
    if (doc[0].lemma_ == 'love' or doc[0].lemma_ == 'hate'):
      if doc[0].lemma_ == 'love':
        return 'love'
      else:
        return 'hate'
    else:
      return None

In [13]:
def exp_3_filtered(s):
  s = preprocess_tweet(s)
  if (filtering(s) == 'good'):
    doc = nlp(s)
    for i in range(len(doc)):
      if doc[i].pos_ == 'VERB':
        if (i != 0) and (doc[i - 1].pos_ == 'ADV') and (doc[i].lemma_ == 'love' or doc[i].lemma_ == 'hate') and [x for x in doc[i].children if x.text.lower() == 'i']:
          if doc[i].lemma_ == 'love':
            return 'love'
          else:
            return 'hate'
        else:
          return None

In [ ]:
only_love_hate["exp1"] = only_love_hate.progress_apply( lambda x : exp_1_filtered(x["tweet"]) , axis = 1 )
only_love_hate.to_csv("new_data.csv", index=False)

100%|██████████| 2824/2824 [45:43<00:00,  1.03it/s]
<ipython-input-87-663ae0178ab0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_love_hate["exp1"] = only_love_hate.progress_apply( lambda x : exp_1_filtered(x["tweet"]) , axis = 1 )


In [ ]:
only_love_hate["exp2"] = only_love_hate.progress_apply( lambda x : exp_2_filtered(x["tweet"]) , axis = 1 )
only_love_hate.to_csv("new_data.csv", index=False)

100%|██████████| 2824/2824 [46:11<00:00,  1.02it/s]
<ipython-input-88-87db4c8dc970>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_love_hate["exp2"] = only_love_hate.progress_apply( lambda x : exp_2_filtered(x["tweet"]) , axis = 1 )


In [ ]:
only_love_hate["exp3"] = only_love_hate.progress_apply( lambda x : exp_3_filtered(x["tweet"]) , axis = 1 )
only_love_hate.to_csv("new_data.csv", index=False)

100%|██████████| 2824/2824 [49:36<00:00,  1.05s/it]
<ipython-input-89-4c80b5f1b8d6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_love_hate["exp3"] = only_love_hate.progress_apply( lambda x : exp_3_filtered(x["tweet"]) , axis = 1 )


In [ ]:
only_love_hate.to_csv("new_data.csv", index=False)

##